In [1]:
import pandas as pd
import numpy as np
import random
from scipy.spatial import distance
import matplotlib.pyplot as plt

# Define parameters
min_distance = 200
tournament_size = 3
target_coverage_area = 800000
turbine_diameter = 126
mutation_rate = 0.015

def initialize_populations():
    return [
        pd.DataFrame({
            'Layout_x': [0, 504, 1008, 252, 756, 1260, 0, 504, 1008, 1512],
            'Layout_y': [0, 0, 0, 252, 252, 252, 504, 504, 504, 504]
        }),
        pd.DataFrame({
            'Layout_x': [0, 378, 756, 252, 630, 1008, 0, 378, 756, 1134],
            'Layout_y': [0, 0, 0, 252, 252, 252, 504, 504, 504, 504]
        }),
        pd.DataFrame({
            'Layout_x': [0, 630, 1260, 315, 945, 1575, 0, 630, 1260],
            'Layout_y': [0, 0, 0, 630, 630, 630, 1260, 1260, 1260]
        })
    ]

def calculate_fitness(df):
    total_coverage_area = len(df) * np.pi * (turbine_diameter / 2)**2
    fitness = total_coverage_area / target_coverage_area

    coordinates = df[['Layout_x', 'Layout_y']].values
    for i in range(len(coordinates)):
        for j in range(i+1, len(coordinates)):
            if distance.euclidean(coordinates[i], coordinates[j]) < min_distance:
                fitness *= 0.9  # Apply penalty

    return fitness

def tournament_selection(populations):
    tournament = random.sample(populations, tournament_size)
    return max(tournament, key=calculate_fitness)

def crossover(parent1, parent2):
    child = parent1.copy()
    for column in child.columns:
        child[column] = np.where(np.random.rand(len(child)) < 0.5, parent1[column], parent2[column])
    return child

def mutate(df):
    for i in range(len(df)):
        if np.random.rand() < mutation_rate:
            for j in range(i+1, len(df)):
                while True:
                    new_x = np.random.rand()
                    new_y = np.random.rand()
                    if distance.euclidean([new_x, new_y], df.loc[j, ['Layout_x', 'Layout_y']]) >= min_distance:
                        df.loc[i, 'Layout_x'] = new_x
                        df.loc[i, 'Layout_y'] = new_y
                        break
    return df

def run_genetic_algorithm(populations):
    for _ in range(100):
        next_gen = []
        for _ in range(len(populations)):
            parent1 = tournament_selection(populations)
            parent2 = tournament_selection(populations)
            child = crossover(parent1, parent2)
            child = mutate(child)
            next_gen.append(child)
        populations = next_gen

    # Sort populations by fitness and return the top 10
    populations.sort(key=calculate_fitness, reverse=True)
    return populations[:10]

def plot_best_layouts(best_layouts):
    fig, axs = plt.subplots(5, 2, figsize=(10, 20))
    for i, layout in enumerate(best_layouts):
        ax = axs[i//2, i%2]
        ax.scatter(layout['Layout_x'], layout['Layout_y'], c='b', s=200)
        ax.set_title(f'Layout {i+1}')
    plt.tight_layout()
    plt.show()

def main():
    populations = initialize_populations()
    best_layouts = run_genetic_algorithm(populations)
    plot_best_layouts(best_layouts)

if __name__ == "__main__":
    main()

/tmp/ipykernel_5327/3280489026.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.520269410142957' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Layout_x'] = new_x
/tmp/ipykernel_5327/3280489026.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9427638870987068' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Layout_y'] = new_y
/tmp/ipykernel_5327/3280489026.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.15750997265434286' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Layout_x'] = new_x
/tmp/ipykernel_5327/3280489026.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will r

KeyboardInterrupt: 